In [8]:
import pandas as pd
import lyricsgenius as lg
import re
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import csv
import urllib.request



In [9]:

# Set up your access token
client_access_token = 'g2sJSzvVjvg1Rm0VAp2ieB1zYVas9BCGZ4VRyZff1dfSXX7jwkke5qJSpMowjUqW'

LyricsGenius = lg.Genius(client_access_token)
song = LyricsGenius.search_song("p!nk", "Just Like a Pill")

Searching for "p!nk" by Just Like a Pill...
Done.


In [10]:
lyrics = song.lyrics
#lyrics = lyrics.split("lyrics", 1)[-1]
lyrics

"24 ContributorsTranslationsΕλληνικάJust Like a Pill Lyrics[Verse 1]\nI'm lyin' here on the floor where you left me\nI think I took too much\nI'm cryin' here, what have you done?\nI thought it would be fun\n\n[Pre-Chorus]\nI can't stay on your life support\nThere's a shortage in the switch\nI can't stay on your morphine\n'Cause it's making me itch\nI said I tried to call the nurse again\nBut she's bein' a little bitch\nI think I'll get outta here\n\n[Chorus]\nWhere I can run, just as fast as I can\nTo the middle of nowhere\nTo the middle of my frustrated fears\nAnd I swear, you're just like a pill\nInstead of making me better\nYou keep making me ill\nYou keep making me ill\n\n[Verse 2]\nI haven't moved from the spot where you left me\nThis must be a bad trip\nAll of the other pills\nThey were different\nMaybe I should get some help\nYou might also like[Pre-Chorus]\nI can't stay on your life support\nThere's a shortage in the switch\nI can't stay on your morphine\n'Cause it's making me 

In [11]:
def clean_text(text):

    text = text.lower()
    text = text.split("lyrics", 1)[-1]
    text = text.replace('\n', ' ')
    text = text.replace('\\', '')
    text = re.sub(r'[\'\",\.!?\\]', '', text)
    text = re.sub(r'\[.*?\]', ' ', text) 
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'[()]', ' ', text)
    
    return text

In [12]:
lyrics = clean_text(lyrics)
lyrics

'  im lyin here on the floor where you left me i think i took too much im cryin here what have you done i thought it would be fun    i cant stay on your life support theres a shortage in the switch i cant stay on your morphine cause its making me itch i said i tried to call the nurse again but shes bein a little bitch i think ill get outta here    where i can run just as fast as i can to the middle of nowhere to the middle of my frustrated fears and i swear youre just like a pill instead of making me better you keep making me ill you keep making me ill    i havent moved from the spot where you left me this must be a bad trip all of the other pills they were different maybe i should get some help you might also like  i cant stay on your life support theres a shortage in the switch i cant stay on your morphine cause its making me itch i said i tried to call the nurse again but shes bein a little bitch i think ill get outta here    where i can run just as fast as i can to the middle of no

In [13]:
# Removing stop words
from gensim.parsing.preprocessing import remove_stopwords
 
# Another sample text
lyrics
 
# Remove stopwords using Gensim
lyrics = remove_stopwords(lyrics)
 
lyrics

'im lyin floor left think took im cryin thought fun stay life support theres shortage switch stay morphine cause making itch said tried nurse shes bein little bitch think ill outta run fast middle middle frustrated fears swear youre like pill instead making better making ill making ill havent moved spot left bad trip pills different maybe help like stay life support theres shortage switch stay morphine cause making itch said tried nurse shes bein little bitch think ill outta run fast middle middle frustrated fears swear youre like pill instead making better making ill making ill run fast middle middle frustrated fears swear youre like pill instead making better making ill making ill stay life support theres shortage switch like pill stay morphine cause making itch like pill said tried nurse shes bein little bitch like pill think ill outta run fast middle middle frustrated fears frustrated fears swear swear youre like pill instead making better making ill making run fast middle middle f

In [14]:
# Setting up model

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [17]:
encoded_input = tokenizer(lyrics, return_tensors='pt', max_length=512, truncation=True, padding=True)
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.5746
2) negative 0.3155
3) positive 0.1099
